In [1]:
import nltk
import numpy as np
import string
from itertools import combinations

In [ ]:
#nltk.download()

In [2]:
from nltk.corpus import brown

In [11]:
#docs in the corpus
brown.fileids()

[u'ca01',
 u'ca02',
 u'ca03',
 u'ca04',
 u'ca05',
 u'ca06',
 u'ca07',
 u'ca08',
 u'ca09',
 u'ca10',
 u'ca11',
 u'ca12',
 u'ca13',
 u'ca14',
 u'ca15',
 u'ca16',
 u'ca17',
 u'ca18',
 u'ca19',
 u'ca20',
 u'ca21',
 u'ca22',
 u'ca23',
 u'ca24',
 u'ca25',
 u'ca26',
 u'ca27',
 u'ca28',
 u'ca29',
 u'ca30',
 u'ca31',
 u'ca32',
 u'ca33',
 u'ca34',
 u'ca35',
 u'ca36',
 u'ca37',
 u'ca38',
 u'ca39',
 u'ca40',
 u'ca41',
 u'ca42',
 u'ca43',
 u'ca44',
 u'cb01',
 u'cb02',
 u'cb03',
 u'cb04',
 u'cb05',
 u'cb06',
 u'cb07',
 u'cb08',
 u'cb09',
 u'cb10',
 u'cb11',
 u'cb12',
 u'cb13',
 u'cb14',
 u'cb15',
 u'cb16',
 u'cb17',
 u'cb18',
 u'cb19',
 u'cb20',
 u'cb21',
 u'cb22',
 u'cb23',
 u'cb24',
 u'cb25',
 u'cb26',
 u'cb27',
 u'cc01',
 u'cc02',
 u'cc03',
 u'cc04',
 u'cc05',
 u'cc06',
 u'cc07',
 u'cc08',
 u'cc09',
 u'cc10',
 u'cc11',
 u'cc12',
 u'cc13',
 u'cc14',
 u'cc15',
 u'cc16',
 u'cc17',
 u'cd01',
 u'cd02',
 u'cd03',
 u'cd04',
 u'cd05',
 u'cd06',
 u'cd07',
 u'cd08',
 u'cd09',
 u'cd10',
 u'cd11',
 u'cd12',


In [212]:
brown.paras('ca19')

[[[u'The', u'Baltimore', u'and', u'Ohio', u'Railroad', u'announced', u'yesterday', u'it', u'would', u'reduce', u'the', u'total', u'amount', u'of', u'its', u'payroll', u'by', u'10', u'per', u'cent', u'through', u'salary', u'cuts', u'and', u'lay-offs', u'effective', u'at', u'12:01', u'A.M.', u'next', u'Saturday', u'.'], [u'The', u'current', u'monthly', u'payroll', u'comes', u'to', u'about', u'$15,000,000', u'.']], [[u'Howard', u'E.', u'Simpson', u',', u'the', u"railroad's", u'president', u',', u'said', u',', u'``', u'A', u'drastic', u'decline', u'in', u'freight', u'loading', u'due', u'principally', u'to', u'the', u'severe', u'slump', u'in', u'the', u'movement', u'of', u'heavy', u'goods', u'has', u'necessitated', u'this', u'regrettable', u'action', u"''", u'.']], ...]

In [52]:
brown.tagged_paras('ca19')[0][0]


[(u'The', u'AT-TL'),
 (u'Baltimore', u'NP'),
 (u'and', u'CC-TL'),
 (u'Ohio', u'NP'),
 (u'Railroad', u'NN-TL'),
 (u'announced', u'VBD'),
 (u'yesterday', u'NR'),
 (u'it', u'PPS'),
 (u'would', u'MD'),
 (u'reduce', u'VB'),
 (u'the', u'AT'),
 (u'total', u'NN'),
 (u'amount', u'VB'),
 (u'of', u'IN'),
 (u'its', u'PP$'),
 (u'payroll', u'NN'),
 (u'by', u'IN'),
 (u'10', u'CD'),
 (u'per', u'IN'),
 (u'cent', u'NN'),
 (u'through', u'IN'),
 (u'salary', u'NN'),
 (u'cuts', u'NNS'),
 (u'and', u'CC'),
 (u'lay-offs', u'NNS'),
 (u'effective', u'JJ'),
 (u'at', u'IN'),
 (u'12:01', u'CD'),
 (u'A.M.', u'RB'),
 (u'next', u'AP'),
 (u'Saturday', u'NR'),
 (u'.', u'.')]

In [40]:
brown.tagged_words('ca19')

[(u'The', u'AT-TL'), (u'Baltimore', u'NP'), ...]

In [233]:
from nltk.corpus import stopwords
stop = stopwords.words('english') + [i for i in string.punctuation]


In [234]:
len({word for word in brown.words('ca19') if word not in stop})

813

In [98]:
len(brown.words('ca19'))
#brown.tagged_paras('ca19')[18][0]

2260

In [199]:
#construct graph
#words = {word[0] for word in brown.tagged_words('ca19') if word[0] not in stop 
#         and word[1] not in ['.', 'NNS', 'CD', ',', '&', 'CC-TL', ')-HL']}
words = {word[0].lower() for word in brown.tagged_words('ca19') if word[0].lower() not in stop 
         and word[1] in ['NN', 'JJ', 'VB']}

G = {word: {word} for word in words}

text = brown.words('ca19')
for i in xrange(1, len(text)):
    w1 = text[i-1].lower()
    w2 = text[i].lower()
    if w1 in G and w2 in G:
        G[w1] = G[w1].union([w2])
        G[w2] = G[w2].union([w1])
scores = {word:1.0 for word in G.keys()}

In [200]:
def score_vertex(G, v, d, scores):
    return (1.0 - d) + d*sum([float(scores[node])/len(G[node]) for node in G[v]]) 

In [205]:
converged = False
while not converged:
    converged = True
    for node in G.keys():
        old_score = scores[node]
        scores[node] = score_vertex(G, node, 0.85, scores)
        if abs(scores[node] - old_score) > 0.0001:
            converged = False
selected = sorted([(scores[node], node) for node in scores.keys()], reverse=True)[:20]
selected = [word[1] for word in selected]

In [206]:
phrases = []
for i in xrange(1, len(text)):
    w1 = text[i-1]
    w2 = text[i]
    if w1 in selected and w2 in selected:
        phrases.append( w1 + " " + w2)
    

In [207]:
phrases

[u'monthly payroll',
 u'union payroll',
 u'individual union',
 u'payroll reduction',
 u'county home',
 u'county school',
 u'brick home',
 u'bedroom floor']

In [208]:
selected

[u'construction',
 u'floor',
 u'payroll',
 u'snow',
 u'business',
 u'total',
 u'home',
 u'county',
 u'condition',
 u'swearing-in',
 u'national',
 u'cent',
 u'housing',
 u'bedroom',
 u'individual',
 u'school',
 u'brick',
 u'reduction',
 u'monthly',
 u'union']

In [188]:
print brown.raw('ca19')



	The/at-tl Baltimore/np and/cc-tl Ohio/np Railroad/nn-tl announced/vbd yesterday/nr it/pps would/md reduce/vb the/at total/nn amount/vb of/in its/pp$ payroll/nn by/in 10/cd per/in cent/nn through/in salary/nn cuts/nns and/cc lay-offs/nns effective/jj at/in 12:01/cd A.M./rb next/ap Saturday/nr ./.
The/at current/jj monthly/jj payroll/nn comes/vbz to/in about/rb $15,000,000/nns ./.


	Howard/np E./np Simpson/np ,/, the/at railroad's/nn$ president/nn ,/, said/vbd ,/, ``/`` A/at drastic/jj decline/nn in/in freight/nn loading/vbg due/jj principally/rb to/in the/at severe/jj slump/nn in/in the/at movement/nn of/in heavy/jj goods/nns has/hvz necessitated/vbn this/dt regrettable/jj action/nn ''/'' ./.


	The/at reduction/nn in/in expenses/nns will/md affect/vb employees/nns in/in the/at thirteen/cd states/nns in/in which/wdt the/at B./np-tl &/cc-tl O./np-tl operates/vbz ./.



Salary/nn-hl cut/nn-hl and/cc-hl lay-offs/nns-hl 
It/pps will/md be/be accomplished/vbn in/in two/cd ways/nns :/: 1/

In [213]:
#Sentence Extraction
#similarity function
def sent_sim(s1, s2):
    common = float(sum([word in s2 for word in s1]))
    if common == 0:
        return 0.0
    else:
        return common/ (np.log(len(s1)) + np.log(len(s2)))

In [287]:
sentences = brown.sents('ca19')
G = np.diag([1.0]*len(sentences))
tagged_sentences = brown.tagged_sents('ca19')
scores = [1.0]*len(sentences)
filtered_sentences = [[word[0] for word in sentence 
                       if word[0] not in stop and word[1] in ['NN', 'JJ', 'VB']] for sentence in tagged_sentences]
#populate graph
for i,j in combinations(range(len(filtered_sentences)), 2):
    G[i][j] = sent_sim(filtered_sentences[i], filtered_sentences[j])

In [288]:
def score_sentence(G, v, d, scores):
    return (1.0 - d) + d*sum([float(scores[i])*G[v][i]/sum(G.T[i]) for i in xrange(len(G[v]))])

converged = False
while not converged:
    converged = True
    for node in xrange(len(sentences)):
        old_score = scores[node]
        scores[node] = score_sentence(G, node, 0.85, scores)
        if abs(scores[node] - old_score) > 0.0001:
            converged = False
            
summary_sents = sorted([index for (score, index) in sorted([(scores[i], i) for i in xrange(len(scores))], reverse=True)[:5]])            


In [317]:
summary_sents = sorted([index for (score, index) in sorted([(scores[i], i) for i in xrange(len(scores))], reverse=True)[:5]])

In [318]:
for sent in summary_sents:
    print ' '.join(sentences[sent])

The Baltimore and Ohio Railroad announced yesterday it would reduce the total amount of its payroll by 10 per cent through salary cuts and lay-offs effective at 12:01 A.M. next Saturday .
Howard E. Simpson , the railroad's president , said , `` A drastic decline in freight loading due principally to the severe slump in the movement of heavy goods has necessitated this regrettable action '' .
A thug struck a cab driver in the face with a pistol last night after robbing him of $18 at Franklin and Mount Streets .
The victim , Norman B. Wiley , 38 , of the 900 block North Charles Street , was treated for cuts at Franklin Square Hospital after the robbery .
A baby was burned to death and two other children were seriously injured last night in a fire which damaged their one-room Anne Arundel county home .


In [319]:
summary_sents

[0, 2, 13, 14, 19]

In [307]:
for sent in sentences:
    print ' '.join(sent)

The Baltimore and Ohio Railroad announced yesterday it would reduce the total amount of its payroll by 10 per cent through salary cuts and lay-offs effective at 12:01 A.M. next Saturday .
The current monthly payroll comes to about $15,000,000 .
Howard E. Simpson , the railroad's president , said , `` A drastic decline in freight loading due principally to the severe slump in the movement of heavy goods has necessitated this regrettable action '' .
The reduction in expenses will affect employees in the thirteen states in which the B. & O. operates .
Salary cut and lay-offs
It will be accomplished in two ways : 1
A flat reduction of 10 per cent in the salary of all officers , supervisors and other employees not belonging to unions .
There are about 3,325 officers and employees in this class .
2
Sufficient lay-offs of union employees to bring about a 10 per cent cut in the union payroll expense .
Since the railroad cannot reduce the salary of individual union members under contract , it m